In [55]:
import re

In [56]:
with open('input19.txt', "r") as file:
        text = file.read().strip().split('\n')

In [57]:
start_c = text[-1]
convs = text[:-2]

In [58]:
convs_dict = {}
for i in convs:
    x, y = i.split(' => ')
    if x in convs_dict:
        list_values = convs_dict[x]
        list_values.extend([y])
        convs_dict[x] = list_values
    else:
        convs_dict[x] = [y]

In [59]:
def generate_single_replacements(string, target, replacements):
    # Find all indices of the target character
    indices = [m.start() for m in re.finditer(target, string)]
    
    result_strings = []
    
    for index in indices:
        for replacement in replacements:
            new_string = list(string)
            new_string[index] = replacement
            result_strings.append(''.join(new_string))
    
    return result_strings

In [60]:
# Example usage
string = "HOH"
target_replacements = {'H': ['OH', 'HO'], 'O' : ['HH']}

all_strings = []
for key in target_replacements:
    new_strings = generate_single_replacements(string, key, target_replacements[key])
    all_strings += new_strings

In [61]:
len(set(all_strings))

4

In [62]:
# Full use
all_strings = []
for key in convs_dict:
    new_strings = generate_single_replacements(start_c, key, convs_dict[key])
    all_strings += new_strings

In [63]:
len(all_strings)

662

In [64]:
len(set(all_strings))

638

In [65]:
def read(s):
    return [i.strip() for i in open(s, 'r')]
lines = read('input19.txt')

replacements = []
for i in lines[:-2]:
    m = re.findall(r'(\S+) => (\S+)', i)
    replacements.append(m[0])
X = lines[-1]

S = set()
for i, j in replacements:
    for k in range(len(X)):
        if X[k:k+len(i)] == i:
            y = X[:k] + j + X[k+len(i):]
            S.add(y)
len(S)

535

In [79]:
replacements = [('e', 'H'),
                ('e', 'O'),
                ('H', 'HO'),
                ('H', 'OH'),
                ('O', 'HH')]
X = 'e'

S = set()
S.add('e')
S_new = set()
iters = 0
while 'HOHOHO' not in S:
    iters += 1
    for X in S:
        for i, j in replacements:
            for k in range(len(X)):
                if X[k:k+len(i)] == i:
                    y = X[:k] + j + X[k+len(i):]
                    S_new.add(y)
    S = S_new.copy()
    print(iters)

{'e'}
{'O', 'H'}
{'O', 'H', 'OH', 'HO', 'HH'}
{'HOH', 'OHH', 'HHO', 'OHO', 'HHH', 'O', 'H', 'OH', 'HOO', 'OOH', 'HO', 'HH'}
{'OHHH', 'HOOO', 'HOH', 'HHO', 'HHHO', 'HHHH', 'OOOH', 'HHH', 'H', 'OHOO', 'OHOH', 'OOHO', 'HHOH', 'HHOO', 'HH', 'OHHO', 'OHH', 'HOOH', 'HOHO', 'OHO', 'O', 'OOHH', 'OH', 'HOO', 'OOH', 'HOHH', 'HO'}
{'OHHH', 'HOOO', 'HOH', 'HHOOO', 'HHHOH', 'HHO', 'HHHO', 'OHHHH', 'HHOOH', 'HOHOO', 'H', 'OHOO', 'OOHHH', 'OOHO', 'OHOHO', 'OHHHO', 'OOHOO', 'HHOO', 'HH', 'OHHOH', 'OHHO', 'OOOOH', 'OHH', 'OOOHH', 'HOHO', 'HHHHO', 'OHO', 'HOOHO', 'OHOHH', 'HOOHH', 'O', 'HOHHH', 'OOHH', 'OH', 'HHHOO', 'HHHH', 'OOOH', 'OHOOH', 'HHH', 'OOHOH', 'OOOHO', 'OHOH', 'HHOH', 'HHOHO', 'HOHOH', 'OHOOO', 'HHOHH', 'HOOH', 'HOOOH', 'HHHHH', 'OOHHO', 'HOOOO', 'HOHHO', 'OHHOO', 'HOO', 'OOH', 'HOHH', 'HO'}
6


In [83]:
replacements = []
for i in lines[:-2]:
    m = re.findall(r'(\S+) => (\S+)', i)
    replacements.append(m[0])
X = lines[-1]

S = set()
S.add('e')
S_new = set()
iters = 0
while X not in S:
    iters += 1
    print(iters)
    for X in S:
        for i, j in replacements:
            for k in range(len(X)):
                if X[k:k+len(i)] == i:
                    y = X[:k] + j + X[k+len(i):]
                    S_new.add(y)
    X = lines[-1]
    S = S_new.copy()
print(iters)

1
2
3
4
5
6
7
8
9
10
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\chris\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\chris\AppData\Local\Temp\ipykernel_10728\1557046627.py", line None, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\chris\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\chris\anaconda3\Lib\site-packages\IPython\core\ultratb.py", line 1435, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\chris\anaconda3\Lib\site-packages\IPython\core\ultratb.py", line 1326, in structured_traceback
    return VerboseTB.structured_traceback(
         

In [85]:
lines[-1]

'CRnCaCaCaSiRnBPTiMgArSiRnSiRnMgArSiRnCaFArTiTiBSiThFYCaFArCaCaSiThCaPBSiThSiThCaCaPTiRnPBSiThRnFArArCaCaSiThCaSiThSiRnMgArCaPTiBPRnFArSiThCaSiRnFArBCaSiRnCaPRnFArPMgYCaFArCaPTiTiTiBPBSiThCaPTiBPBSiRnFArBPBSiRnCaFArBPRnSiRnFArRnSiRnBFArCaFArCaCaCaSiThSiThCaCaPBPTiTiRnFArCaPTiBSiAlArPBCaCaCaCaCaSiRnMgArCaSiThFArThCaSiThCaSiRnCaFYCaSiRnFYFArFArCaSiRnFYFArCaSiRnBPMgArSiThPRnFArCaSiRnFArTiRnSiRnFYFArCaSiRnBFArCaSiRnTiMgArSiThCaSiThCaFArPRnFArSiRnFArTiTiTiTiBCaCaSiRnCaCaFYFArSiThCaPTiBPTiBCaSiThSiRnMgArCaF'

In [92]:
from collections import defaultdict
import re


def parse(stream):
    replacements = defaultdict(list)
    for k, v in re.findall(r"(\w+) => (\w+)", stream):
        replacements[k].append(v)
    return replacements, stream.strip().split("\n")[-1]


def reverse_dict(d):
    reverse = defaultdict(list)
    for k, v in d.items():
        for i in v:
            reverse[i].append(k)
    return reverse


def generate_next(starter, replacements):
    molecules = set()

    for i, char in enumerate(starter):
        try:
            if char in replacements:
                for replacement in replacements[char]:
                    molecules.add(starter[:i] + replacement + starter[i + 1:])
            else:
                for replacement in replacements[starter[i:i + 2]]:
                    molecules.add(starter[:i] + replacement + starter[i + 2:])
        except KeyError:
            continue

    return molecules


def generate_prev(target, replacements):
    molecules = set()

    for k, v in replacements.items():
        idx = target.find(k)
        while idx >= 0:
            for i in v:
                if i == "e":
                    continue
                try:
                    molecules.add(target[:idx] + i + target[idx + len(k):])
                except IndexError:
                    molecules.add(target[:idx] + i)
            idx = target.find(k, idx+1)

    if not molecules:
        molecules = {"e"}
    return molecules


def count_molecules(starter, replacements):
    return len(generate_next(starter, replacements))


def steps_to_generate(target, replacements):
    replacements = reverse_dict(replacements)
    seen = {}
    last_generation = generate_prev(target, replacements)
    n_steps = 1

    while last_generation != {"e"}:
        current_generation = set()
        molecule = min(last_generation, key=len)

        try:
            new_molecules = seen[molecule]
        except KeyError:
            new_molecules = generate_prev(molecule, replacements)
            seen[molecule] = new_molecules
        current_generation |= new_molecules
        last_generation = current_generation

        n_steps += 1

    return n_steps


def part_one():
    with open("input19.txt") as fin:
        replacements, starter = parse(fin.read())
    print(count_molecules(starter, replacements))


def part_two():
    with open("input19.txt") as fin:
        replacements, target = parse(fin.read())
    print(steps_to_generate(target, replacements))


if __name__ == '__main__':
    part_one()
    part_two()

535
212
